# Projeto Introdução a Redes Neurais

**Equipe:**
- Eduardo Luiz
- Ianní
- Tu Chin
- Wellington Barbosa de Almeida - <wba@cin.ufpe.br>

**Professor:** Germano Crispim Vasconcelos

**Objetivo:**
> Realizar um estudo experimental sobre a aplicação de modelos de redes neurais em um problema do mundo real.

**Problema**
> Análise de Risco de Crédito, Com base no perfil de clientes, decidir a quem conceder crédito (risco de inadimplência) 

## Etapa 1 - Divisão da base de dados:

## Base de dados

[Análise de Risco de Crédito](http://www.cin.ufpe.br/~gcv/web_lci/TRN) (+-400K Registros para Treinamento e +-130K Registros para Teste). Tamanho: 731 MB 

Cria a pasta `/data` e realiza o download do dataset na mesma.

In [4]:
%%bash
# criando diretório 'data'
mkdir data
# acessando o novo repositório
cd data
# baixando dataset
curl -O http://www.cin.ufpe.br/~gcv/web_lci/TRN
# descompactando para a pasta 'bens'
#unzip bem_candidato_2014.zip -d bens

Importando os modulos básicos

In [5]:
import numpy as np
import scipy as sp
import pandas as pd

Abrindo e convertendo a base de dados em um [dataframe pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

In [11]:
path = !ls \data
file = open("data/"+path[0],"r")
list_file = file.readlines()

In [14]:
indexes = list_file[0].split('\t') # Pega primeira linha, a linha com os indices, e faz split com \t

indexes[-1] = indexes[-1].replace("\n", "") # remove o '\n' do ultimo arquivo

del list_file[0] # remove os indices para leitura do dataset

data_set = pd.DataFrame.from_records(list_file, columns=indexes)

data_set
#file.close()

MemoryError: 